In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/s7/8db2fs716sj67tmf9cc2npbm0000gn/T/ipykernel_92795/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
from sklearn.model_selection import train_test_split

# Your code
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])
print(f"Training set: {data_train.shape}")
print(f"Validation set: {data_val.shape}")

Training set: (800, 2)
Validation set: (200, 2)


## Data Preprocessing

In [5]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:
# Your code
import re

# Remove JS
def remove_inline_js(text):
	# Remove <script>...</script> blocks (including multiline)
	return re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL | re.IGNORECASE)

data['text_nojs'] = data['text'].apply(remove_inline_js)
data_train['text_nojs'] = data_train['text'].apply(remove_inline_js)
data_val['text_nojs'] = data_val['text'].apply(remove_inline_js)

# Remove CSS
def remove_inline_css(text):
	# Remove <style>...</style> blocks (including multiline)
	return re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL | re.IGNORECASE)

data['text_nocss'] = data['text_nojs'].apply(remove_inline_css)
data_train['text_nocss'] = data_train['text_nojs'].apply(remove_inline_css)
data_val['text_nocss'] = data_val['text_nojs'].apply(remove_inline_css)

# Print the first few rows to verify the result
print(data[['text', 'text_nojs', 'text_nocss']].head())

                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                           text_nojs  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                          text_nocss  
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...  
1                                           Will do.  
2  Nora--Cheryl has emailed dozens of memos about...  
3  Dear Sir=2FMadam=2C I know that this proposal ...  
4                                                fy

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [7]:
import re

# Your code
# Remove all special characters
def remove_special_characters(text):
	return re.sub(r'[^a-zA-Z\s]', '', text)

data['text_nospecial'] = data['text_nocss'].apply(remove_special_characters)
data_train['text_nospecial'] = data_train['text_nocss'].apply(remove_special_characters)
data_val['text_nospecial'] = data_val['text_nocss'].apply(remove_special_characters)

# Remove numbers
def remove_numbers(text):
	return re.sub(r'\d+', '', text)

data['text_nonum'] = data['text_nospecial'].apply(remove_numbers)
data_train['text_nonum'] = data_train['text_nospecial'].apply(remove_numbers)
data_val['text_nonum'] = data_val['text_nospecial'].apply(remove_numbers)

# Remove all single characters
def remove_single_characters(text):
	return re.sub(r'\b[a-zA-Z]\b', '', text)

data['text_nosingle'] = data['text_nonum'].apply(remove_single_characters)
data_train['text_nosingle'] = data_train['text_nonum'].apply(remove_single_characters)
data_val['text_nosingle'] = data_val['text_nonum'].apply(remove_single_characters)

# Remove single characters from the start
def remove_single_characters_start(text):
	return re.sub(r'^[a-zA-Z]\s+', '', text)

data['text_nosingle_start'] = data['text_nosingle'].apply(remove_single_characters_start)
data_train['text_nosingle_start'] = data_train['text_nosingle'].apply(remove_single_characters_start)
data_val['text_nosingle_start'] = data_val['text_nosingle'].apply(remove_single_characters_start)

# Substitute multiple spaces with single space
def substitute_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text).strip()

data['text_nospaces'] = data['text_nosingle_start'].apply(substitute_multiple_spaces)
data_train['text_nospaces'] = data_train['text_nosingle_start'].apply(substitute_multiple_spaces)
data_val['text_nospaces'] = data_val['text_nosingle_start'].apply(substitute_multiple_spaces)

# Remove prefixed 'b'
def remove_prefixed_b(text):
	return re.sub(r'^b\s+', '', text)

data['text_noprefixb'] = data['text_nospaces'].apply(remove_prefixed_b)
data_train['text_noprefixb'] = data_train['text_nospaces'].apply(remove_prefixed_b)
data_val['text_noprefixb'] = data_val['text_nospaces'].apply(remove_prefixed_b)

# Convert to Lowercase
data['preprocessed_text'] = data['text_noprefixb'].str.lower()
data_train['preprocessed_text'] = data_train['text_noprefixb'].str.lower()
data_val['preprocessed_text'] = data_val['text_noprefixb'].str.lower()

print(data[['preprocessed_text']].head())
print(data_train[['preprocessed_text']].head())
print(data_val[['preprocessed_text']].head())

                                   preprocessed_text
0  dear sir strictly private business proposal am...
1                                            will do
2  noracheryl has emailed dozens of memos about h...
3  dear sirfmadamc know that this proposal might ...
4                                                fyi
                                     preprocessed_text
442  dearc good day hope finecdear am writting this...
962  from mr henry kaborethe chief auditor incharge...
971                                            will do
190  from the desk of dradamu ismalerauditing and a...
551  dear friend my name is loi cestradathe wife of...
                                     preprocessed_text
222  strictly personalpermit me to introduce myself...
824  dear nancyi very much want to meet and weve be...
505  request for business partnershipi am patrick a...
165  soneri bank limitedsilver jubilee centercbritt...
832  dear partneri am fund manager with fidelity in...


## Now let's work on removing stopwords
Remove the stopwords.

In [8]:
# Your code
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
	return ' '.join([word for word in text.split() if word not in stop_words])

data['no_stopwords'] = data['preprocessed_text'].apply(remove_stopwords)
data_train['no_stopwords'] = data_train['preprocessed_text'].apply(remove_stopwords)
data_val['no_stopwords'] = data_val['preprocessed_text'].apply(remove_stopwords)

print(data[['preprocessed_text', 'no_stopwords']].head())
print(data_train[['preprocessed_text', 'no_stopwords']].head())
print(data_val[['preprocessed_text', 'no_stopwords']].head())

                                   preprocessed_text  \
0  dear sir strictly private business proposal am...   
1                                            will do   
2  noracheryl has emailed dozens of memos about h...   
3  dear sirfmadamc know that this proposal might ...   
4                                                fyi   

                                        no_stopwords  
0  dear sir strictly private business proposal mi...  
1                                                     
2  noracheryl emailed dozens memos haiti weekend ...  
3  dear sirfmadamc know proposal might surprise e...  
4                                                fyi  
                                     preprocessed_text  \
442  dearc good day hope finecdear am writting this...   
962  from mr henry kaborethe chief auditor incharge...   
971                                            will do   
190  from the desk of dradamu ismalerauditing and a...   
551  dear friend my name is loi cestradathe

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [9]:
# Your code
# Ensure 'no_stopwords' column exists before proceeding
if 'no_stopwords' in data.columns:
	# Break sentences into words
	data['tokens'] = data['no_stopwords'].apply(lambda x: x.split())
	data_train['tokens'] = data_train['no_stopwords'].apply(lambda x: x.split())
	data_val['tokens'] = data_val['no_stopwords'].apply(lambda x: x.split())

	# Use lemmatization (actually stemming here)
	data['lemmatized'] = data['tokens'].apply(lambda tokens: [snowball.stem(token) for token in tokens])
	data_train['lemmatized'] = data_train['tokens'].apply(lambda tokens: [snowball.stem(token) for token in tokens])
	data_val['lemmatized'] = data_val['tokens'].apply(lambda tokens: [snowball.stem(token) for token in tokens])

	# Print the result
	print(data[['no_stopwords', 'tokens', 'lemmatized']].head())
else:
	print("Column 'no_stopwords' not found. Please run the cell that creates it first.")

                                        no_stopwords  \
0  dear sir strictly private business proposal mi...   
1                                                      
2  noracheryl emailed dozens memos haiti weekend ...   
3  dear sirfmadamc know proposal might surprise e...   
4                                                fyi   

                                              tokens  \
0  [dear, sir, strictly, private, business, propo...   
1                                                 []   
2  [noracheryl, emailed, dozens, memos, haiti, we...   
3  [dear, sirfmadamc, know, proposal, might, surp...   
4                                              [fyi]   

                                          lemmatized  
0  [dear, sir, strict, privat, busi, propos, mike...  
1                                                 []  
2  [noracheryl, email, dozen, memo, haiti, weeken...  
3  [dear, sirfmadamc, know, propos, might, surpri...  
4                                              [fyi

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [10]:
from collections import Counter

# Your code
# Flatten lemmatized tokens for ham and spam
ham_words = [word for tokens in data_train[data_train['label'] == 0]['lemmatized'] for word in tokens]
spam_words = [word for tokens in data_train[data_train['label'] == 1]['lemmatized'] for word in tokens]

# Count and get top 10
top_ham = Counter(ham_words).most_common(10)
top_spam = Counter(spam_words).most_common(10)

print("Top 10 words in HAM messages:", top_ham)
print("Top 10 words in SPAM messages:", top_spam)

Top 10 words in HAM messages: [('work', 94), ('call', 92), ('would', 91), ('presid', 91), ('us', 88), ('obama', 79), ('percent', 76), ('state', 72), ('mr', 65), ('time', 63)]
Top 10 words in SPAM messages: [('money', 723), ('bank', 648), ('account', 633), ('fund', 553), ('us', 440), ('transfer', 404), ('foreign', 388), ('busi', 383), ('contact', 357), ('assist', 335)]


## Extra features

In [11]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,text_nojs,text_nocss,text_nospecial,text_nonum,text_nosingle,text_nosingle_start,text_nospaces,text_noprefixb,preprocessed_text,no_stopwords,tokens,lemmatized,money_mark,suspicious_words,text_len
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,Dear=2C Good day hope fine=2Cdear am writting ...,Dear=2C Good day hope fine=2Cdear am writting ...,DearC Good day hope fineCdear am writting this...,DearC Good day hope fineCdear am writting this...,DearC Good day hope fineCdear am writting this...,DearC Good day hope fineCdear am writting this...,DearC Good day hope fineCdear am writting this...,DearC Good day hope fineCdear am writting this...,dearc good day hope finecdear am writting this...,dearc good day hope finecdear writting mail du...,"[dearc, good, day, hope, finecdear, writting, ...","[dearc, good, day, hope, finecdear, writ, mail...",1,1,1538
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,from mr henry kaborethe chief auditor incharge...,mr henry kaborethe chief auditor inchargeforei...,"[mr, henry, kaborethe, chief, auditor, incharg...","[mr, henri, kaboreth, chief, auditor, incharge...",1,1,2910
971,Will do.,0,Will do.,Will do.,Will do,Will do,Will do,Will do,Will do,Will do,will do,,[],[],1,0,7
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,FROM THE DESK OF DRADAMU ISMALERAUDITING AND ...,FROM THE DESK OF DRADAMU ISMALERAUDITING AND ...,FROM THE DESK OF DRADAMU ISMALERAUDITING AND ...,FROM THE DESK OF DRADAMU ISMALERAUDITING AND ...,FROM THE DESK OF DRADAMU ISMALERAUDITING AND A...,FROM THE DESK OF DRADAMU ISMALERAUDITING AND A...,from the desk of dradamu ismalerauditing and a...,desk dradamu ismalerauditing accounting manage...,"[desk, dradamu, ismalerauditing, accounting, m...","[desk, dradamu, ismaleraudit, account, manager...",1,1,497
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,"Dear Friend, My name is LOI C.ESTRADA,The wife...","Dear Friend, My name is LOI C.ESTRADA,The wife...",Dear Friend My name is LOI CESTRADAThe wife of...,Dear Friend My name is LOI CESTRADAThe wife of...,Dear Friend My name is LOI CESTRADAThe wife of...,Dear Friend My name is LOI CESTRADAThe wife of...,Dear Friend My name is LOI CESTRADAThe wife of...,Dear Friend My name is LOI CESTRADAThe wife of...,dear friend my name is loi cestradathe wife of...,dear friend name loi cestradathe wife mr josep...,"[dear, friend, name, loi, cestradathe, wife, m...","[dear, friend, name, loi, cestradath, wife, mr...",1,1,2022


## How would work the Bag of Words with Count Vectorizer concept?

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# Your code
# Use the preprocessed text (without stopwords) for vectorization
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(data_train['no_stopwords'])
X_val_bow = vectorizer.transform(data_val['no_stopwords'])

print("Shape of Bag of Words (CountVectorizer) X_train:", X_train_bow.shape)
print("Shape of Bag of Words (CountVectorizer) X_val:", X_val_bow.shape)

Shape of Bag of Words (CountVectorizer) X_train: (800, 18482)
Shape of Bag of Words (CountVectorizer) X_val: (200, 18482)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [13]:
# Your code
# Load vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Vectorize all dataset
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train['no_stopwords'])
X_val_tfidf = tfidf_vectorizer.transform(data_val['no_stopwords'])

# Print shape
print("Shape of TF-IDF X_train:", X_train_tfidf.shape)
print("Shape of TF-IDF X_val:", X_val_tfidf.shape)

Shape of TF-IDF X_train: (800, 18482)
Shape of TF-IDF X_val: (200, 18482)


## And the Train a Classifier?

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Your code
# Train MultinomialNB classifier using TF-IDF features (as an example of "best" feature representation)
clf = MultinomialNB()
clf.fit(X_train_tfidf, data_train['label'])

# Predict on validation set
y_pred = clf.predict(X_val_tfidf)

# Evaluate
print("Accuracy:", accuracy_score(data_val['label'], y_pred))
print(classification_report(data_val['label'], y_pred))

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [17]:
import numpy as np
from scipy.sparse import hstack
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Your code
# Try combining TF-IDF features with extra features (money_mark, suspicious_words, text_len)

# Select extra features
extra_features_train = data_train[['money_mark', 'suspicious_words', 'text_len']].values
extra_features_val = data_val[['money_mark', 'suspicious_words', 'text_len']].values

# Combine TF-IDF with extra features
X_train_combined = hstack([X_train_tfidf, extra_features_train])
X_val_combined = hstack([X_val_tfidf, extra_features_val])

# Train MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_combined, data_train['label'])

# Predict and evaluate
y_pred = clf.predict(X_val_combined)
print("Accuracy (TF-IDF + extra features):", accuracy_score(data_val['label'], y_pred))
print(classification_report(data_val['label'], y_pred))

Accuracy (TF-IDF + extra features): 0.755
              precision    recall  f1-score   support

           0       1.00      0.56      0.72       112
           1       0.64      1.00      0.78        88

    accuracy                           0.76       200
   macro avg       0.82      0.78      0.75       200
weighted avg       0.84      0.76      0.75       200

